In [16]:

# coding: utf-8

# In[ ]:


from __future__ import print_function

import sys
import numpy as np
from math import sqrt 
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/r09525031/dft/design_1_wrapper.bit")
    dft = ol.dft_0;
    size = 256;
    golden = open("/home/xilinx/r09525031/dft/out.gold.dat", "r+");

    
    in_r = allocate(shape=(size,), dtype=np.float32)
    in_i = allocate(shape=(size,), dtype=np.float32)
    out_r = allocate(shape=(size,), dtype=np.float32)
    out_i = allocate(shape=(size,), dtype=np.float32)
    for i in range(size):
        in_r[i] = i;
    

    timeKernelStart = time()

    dft.write(0x10, in_r.device_address)
    dft.write(0x18, in_i.device_address)
    dft.write(0x20, out_r.device_address)
    dft.write(0x28, out_i.device_address)
    dft.write(0x00, 0x01)
    while (dft.read(0x00) & 0x4) == 0x0:
        continue
    timeKernelEnd = time()
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s")
      
    print("Data check")
    sum_r = 0.0;
    sum_i = 0.0;
    
    for i in range(size):
        data = golden.readline();
        x = data.split();
        gold_r = float(x[1]);
        gold_i = float(x[2]);
        sum_r = sum_r + (gold_r - out_r[i]) * (gold_r - out_r[i]);
        sum_i = sum_i + (gold_i - out_i[i]) * (gold_i - out_i[i]);
        
                  
    error_r = sqrt(sum_r/size);
    error_i = sqrt(sum_i/size);
    print("RMSE(R):" + str(error_r))
    print("RMSE(I):" + str(error_i))
    golden.close();


    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Kernel execution time: 0.06415462493896484 s
Data check
RMSE(R):2.759875892851594e-07
RMSE(I):2.7133535924166183e-07
Exit process
